# Load preprocessed dataset
- AI hub + KorQuAD에 있는 question-answer pair 40만개

In [2]:
import pickle

with open("/content/drive/MyDrive/haystack_tutorial/faq_dataset.pickle", "rb") as f:
    qa_dataset = pickle.load(f)

In [3]:
qa_sets = list()
for q, a in qa_dataset.items():
  pair = dict()
  pair['text'] = q
  pair['answer'] = a
  qa_sets.append(pair)

# Setting FAISS Document Store
- Haystack에서 제공하는 faissdocumentstore 사용
- sql_url을 지정해놓음으로서 재사용시 빠르게 load가능

In [5]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(
    sql_url='sqlite:///faq.db',
    faiss_index_factory_str='HNSW',
)

In [ ]:
document_store.use_windowed_query = False
document_store.write_documents(qa_sets)

# Load Bi-Encoder
- 기존에 학습해놓은 모델을 불러온다.(참고: train_bi_encoder.py)

In [7]:
bi_encoder_path = '/content/drive/MyDrive/haystack_tutorial/senetence_transformers_test/model/training_stsbenchmark_kykim-bert-kor-base-2021-06-19_17-25-20'
from haystack.retriever.dense import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store, 
    embedding_model=bi_encoder_path, 
    use_gpu=True,    model_format='sentence_transformers')

faiss document store에 embedding을 update
- Colab V100 기준 1시간 소요
- index를 저장해서 재사용하면 추후 사용이 편해진다.

In [ ]:
document_store.update_embeddings(retriever)

# Load Cross-Encoder
- 기존에 학습해놓은 모델을 불러온다.(참고: train_cross_encoder.py)

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

cross_encoder_path = '/content/drive/MyDrive/haystack_tutorial/senetence_transformers_test/output/training_stsbenchmark-2021-06-20_12-50-33'

cross_encoder = CrossEncoder(cross_encoder_path)

- 위에서 setting한 모델과 faiss document store를 이용해서 query가 들어오면 유사한 질문을 찾아 답변을 해준다. 
- Cross-Encoder를 통해 retrieved question과 query의 유사도를 다시 체크함으로서 정확도를 높인다.

In [ ]:
from hanspell import spell_checker

def get_answer(query: str):
    query = spell_checker.check(query).checked
    results = retriever.retrieve(query)
    for result in results:
        if result.score < 380:
            break
        else:
            p = cross_encoder.predict([[result.text, query]])
            if p > 0.8:
                return result.meta['answer']
    return 'no answer'
            